<a href="https://colab.research.google.com/github/sgat09/SGAT/blob/master/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING** **LIBRARIES**

In [0]:
import pandas as pd 
import io
import statsmodels.api as sm
import math


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


As the covid-19 epidemic brought the world to a standstill there has been a lot of speculation and discussion as to how the economy will react and how will the major businesses take shape. One major aspect of this is the reaction of the stock market to the news of the epidemic and hence the reaction of the investors to the same.

The sensitivity of the stock market to each and every little piece of information published in the newspapers,tabloids and articles makes it even more interesting to study the same.

Since the outbreak of the disease, the pharmaceutical companies have been expected to take the front seat in driving the world out of this epidemic. There has been a lot of emphasis on research and development of new drugs and vaccines so as to cure and prevent the spread of Covid-19. As a result, efficient investors have been willing and wanting to invest more and more in companies that are avidly contributing in the process of R&D as well as manufacturing of various medicines that have contributed in the treatment of the disease so far.

Two of the most prominent drugs that are helping in the cure of Covid-19 are Hydroxychloroquine and Chloroquine Phosphate. India being the largest manufacturer of these drugs has been in the limelight ever since the discovery. So has the companies that manufactuer these drugs, namely Zydus Wellness Ltd. and IPCA Laboratories.Recently, the ministry of health and family welfare had reportedly placed an order for around 10 crore tables of HCQ with Ipca laboratories and Zydus.

 Laurus Labs Ltd., Mangalam Drugs and Organics Ltd. and Unichem Laboratories emerged as the API exporters of the drug to the world.  

 India exported hydroxychloroquine API in April-January 2019-20 period worth USD 1.22 billion. During the same period exports of formulations made from hydroxychloroquine were at USD 5.50 billion.

 







# **INDIA**



#**UPLOADING DATA:**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving LAUL Historical Data.csv to LAUL Historical Data.csv
Saving Nifty 50 Historical Data.csv to Nifty 50 Historical Data.csv
Saving UNLB Historical Data.csv to UNLB Historical Data.csv
Saving ZYDS Historical Data.csv to ZYDS Historical Data.csv
Saving MADR Historical Data.csv to MADR Historical Data.csv
Saving IPCA Historical Data.csv to IPCA Historical Data.csv


In [0]:

nifty = pd.read_csv(io.BytesIO(uploaded['Nifty 50 Historical Data.csv']))
UNLB = pd.read_csv(io.BytesIO(uploaded['UNLB Historical Data.csv']))
ZYDS = pd.read_csv(io.BytesIO(uploaded['ZYDS Historical Data.csv']))
LAUL = pd.read_csv(io.BytesIO(uploaded['LAUL Historical Data.csv']))
MADR = pd.read_csv(io.BytesIO(uploaded['MADR Historical Data.csv']))
IPCA = pd.read_csv(io.BytesIO(uploaded['IPCA Historical Data.csv']))


#**NIFTY**

In [0]:
nifty ['Date']= pd.to_datetime(nifty['Date'])
nifty.index = nifty['Date']
nifty.sort_index(ascending= False)
nifty['Change %']=list(map(lambda  x:x[:-1],nifty['Change %'].values))
nifty['Change %']=pd.to_numeric(nifty['Change %'], downcast='float')
nifty_postCovid= nifty[:103]
nifty_preCovid =nifty[-102:]
nifty_preCovid['Price'] = nifty_preCovid['Price'].str.replace(',','').astype(float)
nifty_postCovid['Price'] = nifty_postCovid['Price'].str.replace(',','').astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


#**MAKING FUNCTIONS**

In [0]:
def stock(X):
  X['Date']= pd.to_datetime(X['Date'])
  X.index = X['Date']
  X.sort_index(ascending= False)
  return X

In [0]:
def stock_preCovid(X):
  X_preCovid = X[-102:]
  return X_preCovid

In [0]:
def stock_postCovid(X):
  X_postCovid= X[:103]
  return X_postCovid

In [0]:
def beta_preCovid(X,Y):
    tblPreCovid=nifty_preCovid.merge(stock_preCovid(X), left_index=True, right_index=True,suffixes=('_nifty','_'+Y))
    tblPreCovid.reset_index('Date')
    tblPreCovid = tblPreCovid[['Change %_nifty','Change %_'+Y]]
    tblPreCovid['Change %_'+Y]=list(map(lambda  x:x[:-1],tblPreCovid['Change %_'+Y].values))
    tblPreCovid['Change %_'+Y]=pd.to_numeric(tblPreCovid['Change %_'+Y], downcast='float')
    return tblPreCovid

In [0]:
def beta_postCovid(X,Y):
    tblPostCovid=nifty_postCovid.merge(stock_postCovid(X), left_index=True, right_index=True,suffixes=('_nifty','_'+Y))
    tblPostCovid.reset_index('Date')
    tblPostCovid = tblPostCovid[['Change %_nifty','Change %_'+Y]]
    tblPostCovid['Change %_'+Y]=list(map(lambda  x:x[:-1],tblPostCovid['Change %_'+Y].values))
    tblPostCovid['Change %_'+Y]=pd.to_numeric(tblPostCovid['Change %_'+Y], downcast='float')
    return tblPostCovid

In [0]:
def regression_preCovid(X,Y):
  x=beta_preCovid(X,''+Y)['Change %_nifty']
  y=beta_preCovid(X,''+Y)['Change %_'+Y]
  x=sm.add_constant(x)
  model=sm.OLS(y.astype(float), x.astype(float)).fit()
  predictions=model.predict(x)
  results = model.summary().tables[1].as_html()
  results=pd.read_html(results,header=0,index_col=0)[0]
  return results

In [0]:
def regression_postCovid(X,Y):
  x=beta_postCovid(X,''+Y)['Change %_nifty']
  y=beta_postCovid(X,''+Y)['Change %_'+Y]
  x=sm.add_constant(x)
  model=sm.OLS(y.astype(float), x.astype(float)).fit()
  predictions=model.predict(x)
  results = model.summary().tables[1].as_html()
  results=pd.read_html(results,header=0,index_col=0)[0]
  return results

In [0]:
Rm_preCovid = math.log((nifty_preCovid[:1].Price.values[0])/(nifty_preCovid[-1:].Price.values[0]))*100
Rm_preCovid

1.5923151118846044

In [0]:
def Ri_preCovid(X):
  preresult = X[-102:]
  preinitvalue = preresult[:1].Price.values[0]
  prefinalvalue = preresult[-1:].Price.values[0] 
  Ri=math.log(preinitvalue/prefinalvalue)*100
  return Ri

In [0]:
def preabnormal(X,Y):
  betaprecovid = regression_preCovid(X,''+Y).coef.values[1]  
  abnormalpre=Ri_preCovid(X)-(betaprecovid)*(Rm_preCovid)
  return abnormalpre

In [0]:
Rm_postCovid = math.log((nifty_postCovid[:1].Price.values[0])/(nifty_postCovid[-1:].Price.values[0]))*100
Rm_postCovid

-20.043924461007506

In [0]:
def Ri_postCovid(X):
  postresult = X[:103]
  postinitvalue = postresult[:1].Price.values[0]
  postfinalvalue = postresult[-1:].Price.values[0] 
  Ri=math.log(postinitvalue/postfinalvalue)*100
  return Ri

In [0]:
def postabnormal(X,Y):
  betapostcovid = regression_postCovid(X,''+Y).coef.values[1]  
  abnormalpost=Ri_postCovid(X)-(betapostcovid)*(Rm_postCovid)
  return abnormalpost

In [0]:
def mean_pos(P):
  pos =   [x for x in P if x>=0]
  if len(pos)!=0:
    p = sum(pos)/len(pos)
    return p 
  else:
    return 0

In [0]:
def mean_neg(P):
  neg =   [x for x in P if x<0]
  if len(neg)!=0:
    n = sum(neg)/len(neg)
    return n 
  else:
    return 0

In [0]:
def trend(P):
  if mean_pos(P)+mean_neg(P)>0:
    return 'General Trend is Momentum'
  else:
    return 'General Trend is Contrarian'  

#**REGRESSION AND RETURNS:**

In [0]:
#stock(nifty)
stock(UNLB)
stock(ZYDS)
stock(LAUL)
stock(MADR)
stock(IPCA)

,Date,Price,Open,High,Low,Vol.,Change %
Date,,,,,,,
2020-04-30,2020-04-30,"1,618.70","1,600.00","1,629.20","1,565.60",859.36K,1.64%
2020-04-29,2020-04-29,"1,592.60","1,624.00","1,640.00","1,585.00",548.75K,-3.18%
2020-04-28,2020-04-28,"1,644.95","1,673.00","1,690.00","1,628.00",573.56K,-1.49%
2020-04-27,2020-04-27,"1,669.90","1,559.00","1,680.00","1,531.05",1.17M,3.42%
2020-04-24,2020-04-24,"1,614.60","1,533.00","1,629.90","1,520.00",1.12M,5.27%
...,...,...,...,...,...,...,...
2019-07-05,2019-07-05,949.30,949.00,954.85,944.00,85.56K,0.03%
2019-07-04,2019-07-04,949.00,965.00,967.00,941.00,115.58K,-1.07%
2019-07-03,2019-07-03,959.25,971.60,971.80,957.00,94.34K,-0.67%


#**REGRESSION:**

In [0]:
regression_preCovid(UNLB,'UNLB')

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.0250,0.233,-0.107,0.915,-0.487,0.437
Change %_nifty,0.6732,0.231,2.909,0.004,0.214,1.132


In [0]:
regression_postCovid(UNLB,'UNLB')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0959,0.398,0.241,0.81,-0.693,0.885
Change %_nifty,0.6594,0.149,4.432,0.00,0.364,0.954


In [0]:
regression_preCovid(ZYDS,'ZYDS')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0897,0.245,0.366,0.715,-0.396,0.575
Change %_nifty,0.6208,0.243,2.555,0.012,0.139,1.103


In [0]:
regression_postCovid(ZYDS,'ZYDS')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0044,0.208,0.021,0.983,-0.408,0.417
Change %_nifty,0.2457,0.078,3.157,0.002,0.091,0.400


In [0]:
regression_preCovid(LAUL,'LAUL')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0436,0.192,0.227,0.821,-0.337,0.424
Change %_nifty,0.5415,0.191,2.842,0.005,0.163,0.920


In [0]:
regression_postCovid(LAUL,'LAUL')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.5327,0.256,2.084,0.04,0.026,1.040
Change %_nifty,0.7062,0.096,7.386,0.00,0.517,0.896


In [0]:
regression_preCovid(MADR,'MADR')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0267,0.355,0.075,0.940,-0.678,0.731
Change %_nifty,0.6813,0.353,1.933,0.056,-0.018,1.381


In [0]:
regression_postCovid(MADR,'MADR')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.9932,0.345,2.882,0.005,0.310,1.677
Change %_nifty,0.3200,0.129,2.483,0.015,0.064,0.576


In [0]:
regression_preCovid(IPCA,'IPCA')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.2220,0.177,1.257,0.212,-0.128,0.572
Change %_nifty,0.3264,0.175,1.861,0.066,-0.022,0.674


In [0]:
regression_postCovid(IPCA,'IPCA')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.4305,0.300,1.437,0.154,-0.164,1.025
Change %_nifty,0.2843,0.112,2.537,0.013,0.062,0.507


#**TABULAR REPRESENTATION**

In [0]:
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()

Saving tabled1.xlsx to tabled1.xlsx


In [0]:
df3 = pd.read_excel ('tabled1.xlsx')
df3 =pd.DataFrame(df3,columns=['STOCKS_INDIA','BETA','P-VALUE','BETA1','P-VALUE1'])
df3.columns=['STOCKS_INDIA','beta_preCovid','p-value_preCovid','beta_postCovid','p-value_postCovid']
df3["p-value_postCovid"] = df3["p-value_postCovid"].astype(float)
df3

,STOCKS_INDIA,beta_preCovid,p-value_preCovid,beta_postCovid,p-value_postCovid
0,Unichem Laboratories Ltd.,0.6723,0.004,0.6594,0.000
1,Zydus Wellness Ltd.,0.6208,0.012,0.2457,0.002
2,Laurus Labs Ltd.,0.5415,0.005,0.7062,0.000
3,Mangalam Drugs and Organics Ltd.,0.6813,0.056,0.3200,0.015
4,IPCA Laboratories Ltd.,0.3264,0.066,0.2843,0.013


All are significant since p-value is less than 0.1 and can be considered for further analysis.

#**RETURNS:**

#**UNICHEM LABORATORIES**

In [0]:
Ri_preCovid(UNLB)

-4.203928012400685

In [0]:
preabnormal(UNLB,'UNLB')

-5.2758745457214005

In [0]:
Ri_postCovid(UNLB)

-8.83876077395034

In [0]:
postabnormal(UNLB,'UNLB')

4.378203015638009

#**ZYDUS WELLNESS LTD.**

In [0]:
ZYDS['Price'] = ZYDS['Price'].str.replace(',','').astype(float)
#nifty_postCovid['Price'] = nifty_postCovid['Price'].str.replace(',','').astype(float)

In [0]:
Ri_preCovid(ZYDS)

7.304349112072127

In [0]:
preabnormal(ZYDS,'ZYDS')

6.315839890614164

In [0]:
Ri_postCovid(ZYDS)

-5.32232740249538

In [0]:
postabnormal(ZYDS,'ZYDS')

-0.397535162425835

#**LAURUS LABS LTD.**

In [0]:
Ri_preCovid(LAUL)

0.8458225467088076

In [0]:
preabnormal(LAUL,'LAUL')

-0.016416086376705663

In [0]:
Ri_postCovid(LAUL)

40.317504936864914

In [0]:
postabnormal(LAUL,'LAUL')

54.47252439122842

#**MANGALAM DRUGS AND ORGANICS LTD.**

In [0]:
Ri_preCovid(MADR)

-0.33955890011382717

In [0]:
preabnormal(MADR,'MADR')

-1.4244031858408084

In [0]:
Ri_postCovid(MADR)

90.02027586690957

In [0]:
postabnormal(MADR,'MADR')

96.43433169443198

#**IPCA LABORATORIES**

In [0]:
IPCA['Price'] = IPCA['Price'].str.replace(',','').astype(float)
#nifty_postCovid['Price'] = nifty_postCovid['Price'].str.replace(',','').astype(float)

In [0]:
IPCA.dtypes

Date        datetime64[ns]
Price              float64
Open                object
High                object
Low                 object
Vol.                object
Change %            object
dtype: object

In [0]:
Ri_preCovid(IPCA)

20.695290747652354

In [0]:
preabnormal(IPCA,'IPCA')

20.17555909513322

In [0]:
Ri_postCovid(IPCA)

35.830014192577906

In [0]:
postabnormal(IPCA,'IPCA')

41.52850191684234

*

#**TABULAR REPRESENTATION**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving T3(3).xlsx to T3(3).xlsx


In [0]:
df = pd.read_excel('T3(3).xlsx')
df

,STOCKS,NORMAL RETURN_PRE(%),ABNORMAL RETURN_PRE(%),NORMAL RETURN_POST(%),ABNORMAL RETURN_POST(%),MOVEMENT
0,Unichem Laboratories Ltd.,-4.2039,-5.2758,-8.8387,4.3782,Contrarian
1,Zydus Wellness Ltd.,7.3043,6.3158,-5.3223,-0.3975,Contrarian
2,Laurus Labs Ltd.,0.8458,-0.0164,40.3175,54.4725,Contrarian
3,Mangalam Drugs and Organics Ltd.,-0.3395,-1.4244,90.0202,96.4343,Contrarian
4,IPCA Laboratories Ltd.,20.6952,20.1755,35.83,41.5285,Momentum
5,-,-,-,-,-,-
6,NIFTY 50 INDEX,1.5923,-,-20.0439,-,-


In [0]:
df = df[:5]
df = list(df['ABNORMAL RETURN_POST(%)'].astype(float))

In [0]:
mean_pos(df)

49.203375

In [0]:
mean_neg(df)

-0.3975

In [0]:
trend(df)

'General Trend is Momentum'

#**ANALYSIS**

The above table shows us the change in the movement of these stocks pre-covid and during the period of crisis. The general assumption is that with the increase in production and contribution of these companies the past trend will change. This is rightfully true for Unichem Laboratories, Mangalam Drugs and Organics Ltd., Laurus labs ltd. and Zydus Wellness Ltd.  Unichem Laboratories Ltd., Laurus Labs Ltd. and Mangalam Drugs and Organics Ltd. that had a negative return rate pre-covid, now have a postive return rate during the period of crisis. All these companies emerged as winner stock during the crisis. The demand for these stocks increased in the crisis period which might be due to the public's trust on the production capacity of the said companies. Interestingly, Zydus Wellness Ltd. showed a negative return rate in the post-covid period which shows that investors considered it to risky to invest in this company and moved to alternative companies. 

IPCA Laboratories Ltd., on the other hand, have maintained there past trend, which can be considered as ideal and safe investing options by the investors. Strong brand names of these companies and the trust of their investors saved them from the shocks of the crisis.


#**UNITED STATES OF AMERICA**

As the novel coronavirus COVID-19 pandemic continues to spread throughout major cities and countries around the world, there are a few Biotech companies helping to not only treat COVID-19 with breakthrough therapies yet cure the virus with a vaccine. This novel virus and global pandemic require novel therapeutic solutions to combat it. Whether repurposing old drugs, creating new ones through advanced technology and AI, and/or combining both into a super cocktail.

Since the coronavirus epidemic began and expanded from China, Gilead Sciences was the first major company to announce a possible treatment for combating the new coronavirus strain COVID-19. Gilead's treatment, Remdesivir, is an antiviral drug that proved effective in fighting the Ebola virus. So far, Remdesivir has done a good job of eliminating most major symptoms after being administered by Doctors testing this on patients.

Novavax Inc.,a preclinical biotechnology company, announced Feb. 26 that it had several vaccine candidates in preclinical animal studies. By April 8, the company said it had identified a COVID-19 vaccine candidate, and it plans to initiate a Phase I clinical study in mid-May. The first phase of the placebo-controlled study will enroll 130 healthy adults; the first round of data from that study is expected in July. In March the company said it had received $4 million from CEPI to develop a COVID-19 vaccine and that Emergent BioSolutions Inc. would support contract development and manufacturing for the experimental vaccine.

Regeneron's efforts to produce a coronavirus vaccine are similar to those of Gilead Sciences. Regeneron has been building on the knowledge of Ebola treatments in order to combat the COVID-19 virus. They anticipate beginning clinical trials sometime before November.

 Starting Feb. 18, Sanofi is working with BARDA to test a preclinical vaccine candidate for severe acute respiratory syndrome (SARS) for COVID-19 using its recombinant DNA platform. A spokesperson said Sanofi aims to put a vaccine into a Phase 1 clinical trial between March 2021 and August 2021. It announced a separate program with Translate Bio Inc. US:TBIO on March 27 to develop a mRNA vaccine.

 Regeneron and Sanofi said March 16 they had started a Phase 2/3 trial testing Kevzara as a treatment for patients who have been hospitalized with severe COVID-19 infections.

 Vir Biotechnology Inc., has two preclinical treatment candidates, VIR-7831 and VIR-7832, and on May 4 said it had identified a treatment candidate as part of a partnership with Alnylam Pharmaceuticals Inc. US:ALNY. The companies said they aim to file for an investigational new drug application by the end of 2020, before the proposed inhaled treatment or prevention therapy would enter trials.

#**UPLOADING DATA:**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving NASDAQ Composite Historical Data.csv to NASDAQ Composite Historical Data.csv
Saving NVAX Historical Data.csv to NVAX Historical Data.csv
Saving GILD Historical Data.csv to GILD Historical Data.csv
Saving SNY Historical Data.csv to SNY Historical Data.csv
Saving REGN Historical Data.csv to REGN Historical Data.csv
Saving VIR Historical Data.csv to VIR Historical Data.csv


In [0]:
nasdaq = pd.read_csv(io.BytesIO(uploaded['NASDAQ Composite Historical Data.csv']))
NVAX = pd.read_csv(io.BytesIO(uploaded['NVAX Historical Data.csv']))
GILD = pd.read_csv(io.BytesIO(uploaded['GILD Historical Data.csv']))
SNY = pd.read_csv(io.BytesIO(uploaded['SNY Historical Data.csv']))
REGN = pd.read_csv(io.BytesIO(uploaded['REGN Historical Data.csv']))
VIR = pd.read_csv(io.BytesIO(uploaded['VIR Historical Data.csv']))

#**NASDAQ**

In [0]:
nasdaq ['Date']= pd.to_datetime(nasdaq['Date'])
nasdaq.index = nasdaq['Date']
nasdaq.sort_index(ascending= False)
nasdaq['Change %']=list(map(lambda  x:x[:-1],nasdaq['Change %'].values))
nasdaq['Change %']=pd.to_numeric(nasdaq['Change %'], downcast='float')
nasdaq_postCovid= nasdaq[:104]
nasdaq_preCovid =nasdaq[-107:]
nasdaq_preCovid['Price'] = nasdaq_preCovid['Price'].str.replace(',','').astype(float)
nasdaq_postCovid['Price'] = nasdaq_postCovid['Price'].str.replace(',','').astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [0]:
nasdaq_preCovid

,Date,Price,Open,High,Low,Vol.,Change %
Date,,,,,,,
2019-11-29,2019-11-29,8665.47,"8,682.01","8,697.33","8,664.03",303.12M,-0.46
2019-11-27,2019-11-27,8705.17,"8,669.59","8,705.91","8,662.58",436.37M,0.66
2019-11-26,2019-11-26,8647.93,"8,635.40","8,659.73","8,625.62",739.47M,0.18
2019-11-25,2019-11-25,8632.49,"8,559.65","8,633.15","8,559.65",575.98M,1.32
2019-11-22,2019-11-22,8519.89,"8,530.54","8,535.46","8,477.49",459.67M,0.16
...,...,...,...,...,...,...,...
2019-07-08,2019-07-08,8098.38,"8,112.91","8,112.91","8,078.39",511.73M,-0.78
2019-07-05,2019-07-05,8161.79,"8,123.28","8,171.97","8,093.66",425.69M,-0.10
2019-07-03,2019-07-03,8170.23,"8,129.57","8,170.23","8,122.34",408.20M,0.75


In [0]:
nasdaq_postCovid

,Date,Price,Open,High,Low,Vol.,Change %
Date,,,,,,,
2020-04-30,2020-04-30,8889.55,"8,911.01","8,926.11","8,825.83",1.13B,-0.28
2020-04-29,2020-04-29,8914.71,"8,802.70","8,957.26","8,765.01",1.03B,3.57
2020-04-28,2020-04-28,8607.73,"8,825.68","8,830.57","8,600.70",846.77M,-1.40
2020-04-27,2020-04-27,8730.16,"8,717.98","8,754.57","8,697.37",807.92M,1.11
2020-04-24,2020-04-24,8634.52,"8,530.08","8,642.93","8,464.42",797.88M,1.65
...,...,...,...,...,...,...,...
2019-12-06,2019-12-06,8656.53,"8,634.25","8,665.44","8,630.58",527.65M,1.00
2019-12-05,2019-12-05,8570.70,"8,587.93","8,588.89","8,541.92",536.69M,0.05
2019-12-04,2019-12-04,8566.67,"8,557.44","8,584.88","8,552.38",571.97M,0.54


#**MAKING FUNCTIONS:**

In [0]:
def stocku2(X):
  X['Date']= pd.to_datetime(X['Date'])
  X.index = X['Date']
  X.sort_index(ascending= False)
  return X

In [0]:
def stock_preCovidu2(X):
  X_preCovid = X[-107:]
  return X_preCovid

In [0]:
def stock_postCovidu2(X):
  X_postCovid= X[:104]
  return X_postCovid

In [0]:
def beta_preCovidu2(X,Y):
    tblPreCovidu=nasdaq_preCovid.merge(stock_preCovidu2(X), left_index=True, right_index=True,suffixes=('_nasdaq','_'+Y))
    tblPreCovidu.reset_index('Date')
    tblPreCovidu = tblPreCovidu[['Change %_nasdaq','Change %_'+Y]]
    tblPreCovidu['Change %_'+Y]=list(map(lambda  x:x[:-1],tblPreCovidu['Change %_'+Y].values))
    tblPreCovidu['Change %_'+Y]=pd.to_numeric(tblPreCovidu['Change %_'+Y], downcast='float')
    return tblPreCovidu

In [0]:
def beta_postCovidu2(X,Y):
    tblPostCovidu=nasdaq_postCovid.merge(stock_postCovidu2(X), left_index=True, right_index=True,suffixes=('_nasdaq','_'+Y))
    tblPostCovidu.reset_index('Date')
    tblPostCovidu = tblPostCovidu[['Change %_nasdaq','Change %_'+Y]]
    tblPostCovidu['Change %_'+Y]=list(map(lambda  x:x[:-1],tblPostCovidu['Change %_'+Y].values))
    tblPostCovidu['Change %_'+Y]=pd.to_numeric(tblPostCovidu['Change %_'+Y], downcast='float')
    return tblPostCovidu

In [0]:
def regression_preCovidu2(X,Y):
  x=beta_preCovidu2(X,''+Y)['Change %_nasdaq']
  y=beta_preCovidu2(X,''+Y)['Change %_'+Y]
  x=sm.add_constant(x)
  model=sm.OLS(y.astype(float), x.astype(float)).fit()
  predictions=model.predict(x)
  results = model.summary().tables[1].as_html()
  results=pd.read_html(results,header=0,index_col=0)[0]
  return results

In [0]:
def regression_postCovidu2(X,Y):
  x=beta_postCovidu2(X,''+Y)['Change %_nasdaq']
  y=beta_postCovidu2(X,''+Y)['Change %_'+Y]
  x=sm.add_constant(x)
  model=sm.OLS(y.astype(float), x.astype(float)).fit()
  predictions=model.predict(x)
  results = model.summary().tables[1].as_html()
  results=pd.read_html(results,header=0,index_col=0)[0]
  return results

In [0]:
Rm_preCovidu2 = math.log((nasdaq_preCovid[:1].Price.values[0])/(nasdaq_preCovid[-1:].Price.values[0]))*100
Rm_preCovidu2

6.8574055207913585

In [0]:
def Ri_preCovidu2(X):
  preresult = X[-107:]
  preinitvalue = preresult[:1].Price.values[0]
  prefinalvalue = preresult[-1:].Price.values[0] 
  Ri=math.log(preinitvalue/prefinalvalue)*100
  return Ri

In [0]:
def preabnormalu2(X,Y):
  betaprecovidu2 = regression_preCovidu2(X,''+Y).coef.values[1]  
  abnormalpreu=Ri_preCovidu2(X)-(betaprecovidu2)*(Rm_preCovidu2)
  return abnormalpreu

In [0]:
Rm_postCovidu2 = math.log((nasdaq_postCovid[:1].Price.values[0])/(nasdaq_postCovid[-1:].Price.values[0]))*100
Rm_postCovidu2

3.684326356073244

In [0]:
def Ri_postCovidu2(X):
  postresult = X[:104]
  postinitvalue = postresult[:1].Price.values[0]
  postfinalvalue = postresult[-1:].Price.values[0] 
  Ri=math.log(postinitvalue/postfinalvalue)*100
  return Ri

In [0]:
def postabnormalu2(X,Y):
  betapostcovidu2 = regression_postCovidu2(X,''+Y).coef.values[1]  
  abnormalpostu=Ri_postCovidu2(X)-(betapostcovidu2)*(Rm_postCovidu2)
  return abnormalpostu

#**REGRESSIONS AND RETURNS:**

In [0]:
stocku2(NVAX)
stocku2(VIR)
stocku2(REGN)
stocku2(SNY)
stocku2(GILD)
#stocku2(nasdaq)


,Date,Price,Open,High,Low,Vol.,Change %
Date,,,,,,,
2020-04-30,2020-04-30,84.00,85.36,85.36,81.36,39.33M,1.03%
2020-04-29,2020-04-29,83.14,84.09,85.00,80.05,78.23M,5.68%
2020-04-28,2020-04-28,78.67,80.00,80.00,78.09,9.78M,-1.54%
2020-04-27,2020-04-27,79.90,80.31,80.35,78.25,14.64M,0.33%
2020-04-24,2020-04-24,79.64,77.10,80.15,76.25,24.44M,2.39%
...,...,...,...,...,...,...,...
2019-07-08,2019-07-08,67.48,68.27,68.47,66.36,7.19M,-1.53%
2019-07-05,2019-07-05,68.53,68.56,69.34,68.35,4.12M,-0.68%
2019-07-03,2019-07-03,69.00,69.07,69.07,68.32,2.91M,0.23%


#**REGRESSIONS**

In [0]:
regression_preCovidu2(NVAX,'NVAX')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0572,0.643,0.089,0.929,-1.218,1.333
Change %_nasdaq,-0.4890,0.650,-0.752,0.454,-1.778,0.800


In [0]:
regression_postCovidu2(NVAX,'NVAX')

,coef,std err,t,P>|t|,[0.025,0.975]
const,1.8846,1.190,1.583,0.116,-0.476,4.245
Change %_nasdaq,0.6855,0.401,1.711,0.090,-0.109,1.480


In [0]:
regression_preCovidu2(VIR,'VIR')

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.9102,0.547,-1.663,0.106,-2.025,0.204
Change %_nasdaq,2.2783,0.923,2.468,0.019,0.398,4.158


In [0]:
regression_postCovidu2(VIR,'VIR')

,coef,std err,t,P>|t|,[0.025,0.975]
const,1.5680,1.208,1.298,0.197,-0.828,3.964
Change %_nasdaq,-0.4428,0.407,-1.089,0.279,-1.249,0.364


In [0]:
regression_preCovidu2(REGN,'REGN')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0995,0.157,0.633,0.528,-0.212,0.411
Change %_nasdaq,0.8973,0.159,5.651,0.000,0.582,1.212


In [0]:
regression_postCovidu2(REGN,'REGN')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.3469,0.251,1.383,0.17,-0.150,0.844
Change %_nasdaq,0.5821,0.084,6.897,0.00,0.415,0.749


In [0]:
regression_preCovidu2(SNY,'SNY')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.0447,0.110,0.405,0.686,-0.174,0.263
Change %_nasdaq,0.4237,0.112,3.800,0.000,0.203,0.645


In [0]:
regression_postCovidu2(SNY,'SNY')

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.0076,0.177,-0.043,0.966,-0.36,0.345
Change %_nasdaq,0.6483,0.060,10.851,0.000,0.53,0.767


In [0]:
regression_preCovidu2(GILD,'GILD')

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.0523,0.107,-0.488,0.627,-0.265,0.160
Change %_nasdaq,0.7111,0.108,6.571,0.000,0.497,0.926


In [0]:
regression_postCovidu2(GILD,'GILD')

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.2279,0.273,0.834,0.406,-0.314,0.770
Change %_nasdaq,0.4954,0.092,5.385,0.000,0.313,0.678


#**TABULAR REPRESENTATION**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving T2.xlsx to T2.xlsx


In [0]:
df4 = pd.read_excel ('T2.xlsx')
df4 =pd.DataFrame(df4,columns=['STOCKS','BETA','P-VALUE','BETA1','P-VALUE1'])
df4.columns=['STOCKS_US','beta_preCovid','p-value_preCovid','beta_postCovid','p-value_postCovid']
df4["p-value_postCovid"] = df4["p-value_postCovid"].astype(float)
df4

,STOCKS_US,beta_preCovid,p-value_preCovid,beta_postCovid,p-value_postCovid
0,Novavax Inc.,-0.0489,0.454,0.6855,0.090
1,Vir Biotechnology Inc.,2.2783,0.019,-0.4428,0.279
2,Regeneron Pharmaceuicals Inc.,0.8973,0.000,0.5821,0.000
3,Sanofi ADR,0.4237,0.000,0.6483,0.000
4,Gilead Sciences Inc.,0.7111,0.000,0.4954,0.000


Since p-value for Novavax Inc. and Vir Biotechnology is greater than 0.1 these stcoks are considered to be insignificant and  FURTHER ANALYSIS for these will not take place.

#**RETURNS**

#**Regeneron Pharmaceuticals Inc.**

In [0]:
Ri_preCovidu2(REGN)

15.328384696335823

In [0]:
preabnormalu2(REGN,'REGN')

9.175234722529737

In [0]:
Ri_postCovidu2(REGN)

36.222116107702526

In [0]:
postabnormalu2(REGN,'REGN')

34.07746973583229

#**Sanofi ADR**

In [0]:
Ri_preCovidu2(SNY)

6.894696728900455

In [0]:
preabnormalu2(SNY,'SNY')

3.9892140097411564

In [0]:
Ri_postCovidu2(SNY)

2.1803346170100784

In [0]:
postabnormalu2(SNY,'SNY')

-0.20821415963220602

#**Gilead Sciences Inc.**

In [0]:
Ri_preCovidu2(GILD)

-0.8736266417573326

In [0]:
preabnormalu2(GILD,'GILD')

-5.749927707592067

In [0]:
Ri_postCovidu2(GILD)

23.934552589049016

In [0]:
postabnormalu2(GILD,'GILD')

22.109337312250332

#**TABLE**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving T4(2).xlsx to T4(2).xlsx


In [0]:
df2 = pd.read_excel ('T4(2).xlsx')
df2


,STOCKS_USA,NORMAL RETURN_PRE(%),ABNORMAL RETURN_PRE(%),NORMAL RETURN_POST(%),ABNORMAL RETURN_POST(%),MOVEMENT
0,Regeneron Pharmaceuticals Ltd.,15.3283,9.1752,36.2221,34.0774,Momentum
1,Sanofi ADR,6.8946,3.9892,2.1803,-0.2082,Contrarian
2,Gilead Sciences Inc.,-0.8736,-5.7499,23.9345,22.1093,Contrarian
3,-,-,-,-,-,-
4,NASDAQ COMPOSITE,6.8574,-,3.6843,-,-


In [0]:
df2 = df2[:3]
df2 = list(df2['ABNORMAL RETURN_POST(%)'].astype(float))

In [0]:
mean_pos(df2)

28.09335

In [0]:
mean_neg(df2)

-0.2082

In [0]:
trend(df2)

'General Trend is Momentum'

#**ANALYSIS**

The above companies seem like a trustworthy hedge for the uncertainty COVID-19 brings to the rest of the financial market. However, experts are warning that creating and approving a vaccine isn't a straight road and might take another few years. Even then, efforts must be coordinated on a global scale, a very difficult task. This can be evidently seen in investor's trust in Sanofi ADR, one of the three companies chosen for further analysis. The contrarian movement from a positive return in the pre-covid period to a negative one ine the post covid period shows the uncertainty and risks associated with the innovation and research of these companies. Gilead Sciences Inc. that was facing a negative return in the pre-covid period, due to its extensive research and successes earned the trust of its investors, leading to a positive return in the post-covid period. Regeneron Pharmaceuticals Ltd. saw a momentum movement, which depicts that it persisted the shocks related to the spread of Covid-19 in a proper way.

#**REFERENCES**

https://www.marketwatch.com/press-release/these-companies-are-high-potential-game-changers-for-covid-19-treatments-vaccines-2020-03-27

https://www.marketwatch.com/story/these-nine-companies-are-working-on-coronavirus-treatments-or-vaccines-heres-where-things-stand-2020-03-06

https://www.businesstoday.in/sectors/pharma/hydroxychloroquine-makers-zydus-ipca-others-plan-6-times-capacity-to-meet-demand/story/400419.html

https://theprint.in/world/indian-firms-ipca-zydus-cadila-get-big-us-orders-for-chloroquine-to-fight-covid-19/385859/

https://pharmaboardroom.com/articles/covid-19-a-material-disruptor/

https://timesofindia.indiatimes.com/business/india-business/enough-capacity-in-country-to-meet-hydroxychloroquine-demand-industry/articleshow/75029934.cms


